In [1]:
# 01 - Import Libraries
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import PIL
from torch.utils.data import DataLoader
from timeit import default_timer as timer
from tqdm.auto import tqdm
RANDOM_SEED = torch.manual_seed(42)

In [2]:
# Necessary Functions
device = "cuda" if torch.cuda.is_available() else "cpu"
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc
def train_timer(start:float, end:float, device: torch.device=None):
    total_time = end-start
    print(f"Device:{device}:{total_time:.2f}sec")
    return total_time

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    model.eval()
    total_loss, total_acc = 0, 0

    with torch.inference_mode():
        for img, labels in tqdm(data_loader):
            img, labels = img.to(device), labels.to(device)  
            preds = model(img)
            loss = loss_fn(preds, labels)
            acc = accuracy_fn(labels, preds.argmax(dim=1))

            total_loss += loss.item()
            total_acc += acc

    avg_loss = total_loss / len(data_loader)
    avg_acc = total_acc / len(data_loader)

    return {
        "model_name": model.__class__.__name__,
        "model_loss": avg_loss,
        "model_acc": avg_acc
    }

In [3]:
# 02 - Datasets
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform= None,
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform= None,
)

In [4]:
# 03 - DataLoad
class_names = train_data.classes
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=32,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=32,
                              shuffle=False)
train_features_batch, train_labels_batch = next(iter(train_dataloader))


In [ ]:
# 04 - Non-Linear Model 
class FashionMNISTModelV1(nn.Module):
    def __init__(self, input_shape:int, hidden_units:int, output_shape:int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU())
    def forward(self, x):
        return self.layer_stack(x)

model_1 = FashionMNISTModelV1(
    input_shape=28*28,
    hidden_units=10,
    output_shape=len(class_names)
)
model_1

FashionMNISTModelV1(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): ReLU()
  )
)

In [6]:
# 05 - Loss Fnc. & Optimizer
loss_fn1 = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)

In [7]:
# 06 - Training & Testing
RANDOM_SEED
train_time_start = timer()
epochs = 3

for epoch in range(epochs):
    print(f"Epoch: {epoch + 1}")
    model_1.train()
    epoch_loss = 0

    for batch, (img, labels) in enumerate(tqdm(train_dataloader)):
        img, labels = img.to(device), labels.to(device)

        # Forward pass
        train_preds = model_1(img)
        train_loss = loss_fn1(train_preds, labels)
        epoch_loss += train_loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if batch % 400 == 0:
            seen = batch * len(img)
            total = len(train_dataloader.dataset)
            print(f"Looked at: {seen}/{total} samples")

    epoch_loss /= len(train_dataloader)

    # Testing
    model_1.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for img, labels in test_dataloader:
            img, labels = img.to(device), labels.to(device)

            test_preds = model_1(img)
            test_loss += loss_fn1(test_preds, labels).item()
            test_acc += accuracy_fn(labels, test_preds.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

    print(f"Train Loss: {epoch_loss:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

train_time_end = timer()
total_train_time_model_1 = train_timer(train_time_start, train_time_end, device=device)

Epoch: 1


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.7664 | Test Loss: 0.4958 | Test Acc: 82.7476
Epoch: 2


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.4738 | Test Loss: 0.4843 | Test Acc: 82.6478
Epoch: 3


  0%|          | 0/1875 [00:00<?, ?it/s]

Looked at: 0/60000 samples
Looked at: 12800/60000 samples
Looked at: 25600/60000 samples
Looked at: 38400/60000 samples
Looked at: 51200/60000 samples
Train Loss: 0.4357 | Test Loss: 0.4638 | Test Acc: 83.3067
Device:cpu:4208.89sec


In [ ]:
# 07 - Result Evaluation
RANDOM_SEED
model_1_results = eval_model(model_1, test_dataloader, loss_fn1, accuracy_fn)
model_1_results


  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name': 'FashionMNISTModelV1',
 'model_loss': 0.4638106187644858,
 'model_acc': 83.30670926517571}

In [ ]:
# 08 - Saving the model
torch.save(model_1.state_dict(), "Models/model_1.pth")
print("Model saved to model_1.pth")

Model saved to model_1.pth
